### Explainability analysis with SHAP for selected features.
This script uses the features selected from the classification pipeline to train a Linear SVM classifier and analyze model explainability
via SHAP values.

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.calibration import CalibratedClassifierCV
from numpy import mean, std

import shap
from matplotlib.colors import ListedColormap

In [ ]:
# Load data
# ----------------------------
# Path to the Excel file (update before running)
path_to_excel_file = "path/to/data.xlsx"

# Read dataset
df = pd.read_excel(path_to_excel_file)

# Keep only diagnostic groups of interest
df = df[df["Group"].isin([2, 3, 4, 5])]

In [ ]:
# Define features and labels
# ----------------------------
X = df.drop(columns=["Group"])
y = df["Group"]

In [ ]:
# Features previously selected (from feature selection pipeline)
selected_features = [
    "best_feature_1",
    "best_feature_2",
    "best_feature_3",
]  # Update with actual feature names
X_best = X[selected_features]

In [ ]:
# Model training and evaluation

scaler = StandardScaler()
model = LinearSVC(C=0.25)
cv = LeaveOneOut()

# Scale features
X_scaled = scaler.fit_transform(X_best)

# Train model and evaluate with LOOCV
model.fit(X_scaled, y)
scores = cross_val_score(model, X_scaled, y, scoring="accuracy", cv=cv, n_jobs=-1)
print(f"Accuracy: {mean(scores):.4f} ± {std(scores):.4f}")

In [ ]:
# SHAP Analysis

# Calibrate model to output probabilities (needed for SHAP)
clf = CalibratedClassifierCV(model, cv="prefit")
clf.fit(X_scaled, y)

In [ ]:
# Compute SHAP values using KernelExplainer
explainer = shap.KernelExplainer(clf.predict_proba, X_scaled)
shap_values = explainer.shap_values(X_scaled)

# Define class names (replace with actual labels if needed)
class_names = ["Class 1", "Class 2", "Class 3"]

shap_features_names = selected_features  # Update with actual feature names if needed

In [ ]:
# Plot SHAP summary (bar plot, all classes)

shap.summary_plot(
    shap_values,
    X_scaled,
    plot_type="bar",
    class_names=class_names,
    feature_names=shap_features_names,
    show=False,
    max_display=30,
)

In [ ]:
fig, ax = plt.gcf(), plt.gca()
ax.tick_params(labelsize=12)
ax.set_xlabel("SHAP value (impact on model output)", fontsize=12)
ax.set_title("Feature Importance - Multiclass SVC model", fontsize=16)

plt.savefig("shap_summary.png", bbox_inches="tight", dpi=300)

In [ ]:
# Plot SHAP summary (violin plot, one class vs rest)
# ----------------------------
shap.summary_plot(
    shap_values[0],  # class index 0 = "Class 1"
    X_scaled,
    plot_type="violin",
    class_names=class_names,
    feature_names=shap_features_names,
    max_display=10,
    show=False,
)

fig, ax = plt.gcf(), plt.gca()
ax.set_title("Feature Importance - Multiclass SVC model, Class 1 vs rest", fontsize=14)

plt.savefig("shap_class1.png", bbox_inches="tight", dpi=300)